## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import time
import os

from tqdm import tqdm

import boto3
from langchain.prompts import PromptTemplate
from langchain_aws import ChatBedrock

from IPython.display import display, HTML

## Data Display

In [ ]:
def data_display(data):
    # Display the DataFrame with scroll and define the height and width for the scrollable area
    display(HTML(f'''
    <div style="height: 500px; overflow-y: scroll; overflow-x: scroll; border: 1px solid black; padding: 5px;">
        {data.to_html(max_rows=None, max_cols=None)}
    </div>
    '''))

## Function: LLM prediction for General User Case

In [ ]:
## Function to perform prediction with an LLM for the General User Case with retry logic
def get_prediction_GeneralUser(row, chain, max_retries=5, initial_wait=1):
    hpi = row['HPI']
    patient_info = row["patient_info"]
    attempt = 0
    while attempt < max_retries:
        try:
            # Invoke the chain with the HPI and patient_info
            response = chain.invoke({"HPI": hpi, "patient_info": patient_info}).content
            return response  # Return on successful invocation

        except Exception as e:
            # Check if the error is a ThrottlingException or similar
            if "ThrottlingException" in str(e) or "Too many requests" in str(e):
                # Exponential backoff
                wait_time = initial_wait * (2 ** attempt)
                print(f"Throttling detected. Retrying after {wait_time} seconds...")
                time.sleep(wait_time)
                attempt += 1
            else:
                # Handle other types of exceptions
                return f"Error: {str(e)}"
    # If all retries fail, return an error
    return "Error: Max retries exceeded"

## Function: LLM prediction for Clincial User Case

In [ ]:
## Function to perform prediction with an LLM for the Clinical User Case with retry logic
def get_prediction_ClinicalUser(row, chain, max_retries=5, initial_wait=1):
    hpi = row['HPI']
    patient_info = row["patient_info"]
    initial_vitals = row["initial_vitals"]
    attempt = 0
    while attempt < max_retries:
        try:
            # Invoke the chain with the HPI, patient_info and initial_vitals
            response = chain.invoke({"hpi": hpi, "patient_info": patient_info, "initial_vitals": initial_vitals}).content
            return response  # Return on successful invocation
        
        except Exception as e:
            # Check if the error is a ThrottlingException or similar
            if "ThrottlingException" in str(e) or "Too many requests" in str(e):
                # Exponential backoff
                wait_time = initial_wait * (2 ** attempt)
                print(f"Throttling detected. Retrying after {wait_time} seconds...")
                time.sleep(wait_time)
                attempt += 1
            else:
                # Handle other types of exceptions
                return f"Error: {str(e)}"
    # If all retries fail, return an error
    return "Error: Max retries exceeded"


## LLM Prediction Specialty and Diagnosis General User Case

In [ ]:
## Load Data from create_ground_truth_specialty.ipynb
df = pd.read_csv("MIMIC-IV-Ext-Diagnosis-Specialty.csv")

In [ ]:
## Define the prompt template
prompt = """You are an experienced healthcare professional with expertise in determining the medical specialty and diagnosis based on a patient's history of present illness and personal information. Review the data and identify the three most likely, distinct specialties to manage the condition, followed by the three most likely diagnoses. List specialties first, in order of likelihood, then diagnoses. 
Respond with the specialties in <specialty> tags and the diagnoses in <diagnosis> tags.
History of present illness: {hpi} and personal information: {patient_info}."""


## set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"]="Enter your AWS Access Key ID"
os.environ["AWS_SECRET_ACCESS_KEY"]="Enter your AWS Secret Access Key"

prompt_chain = PromptTemplate(template=prompt,input_variables=["hpi", "patient_info"])
client = boto3.client(service_name="bedrock-runtime", region_name=str("us-east-1"))


## Claude Sonnet 3.5
llm_claude35 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={"temperature": 0},client=client)
chain_claude35 = prompt_chain | llm_claude35


## Claude Sonnet 3
llm_claude3 = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_claude3 = prompt_chain | llm_claude3


## Claude 3 Haiku
llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_haiku = prompt_chain | llm_haiku


tqdm.pandas()
df['diag_spec_Claude3.5'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_claude35), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

df['diag_spec_Claude3'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_claude3), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

df['diag_spec_Haiku'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_haiku), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

## LLM Prediction Specialty and Diagnosis Clinical User Case

In [ ]:
## Define the prompt template
prompt = """You are an experienced healthcare professional with expertise in determining the medical specialty and diagnosis based on a patient's history of present illness, personal information and initial vitals. Review the data and identify the three most likely, distinct specialties to manage the condition, followed by the three most likely diagnoses. List specialties first, in order of likelihood, then diagnoses.
Respond with the specialties in <specialty> tags and the diagnoses in <diagnosis> tags.
History of present illness: {hpi}, personal information: {patient_info} and initial vitals: {initial_vitals}."""


## set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"]="Enter your AWS Access Key ID"
os.environ["AWS_SECRET_ACCESS_KEY"]="Enter your AWS Secret Access Key"

prompt_chain = PromptTemplate(template=prompt,input_variables=["hpi", "patient_info", "initial_vitals"])
client = boto3.client(service_name="bedrock-runtime", region_name=str("us-east-1"))


## Claude Sonnet 3.5
llm_claude35 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={"temperature": 0},client=client)
chain_claude35 = prompt_chain | llm_claude35


## Claude Sonnet 3
llm_claude3 = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_claude3 = prompt_chain | llm_claude3


## Claude 3 Haiku
llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_haiku = prompt_chain | llm_haiku


tqdm.pandas()
df['diag_spec_Claude3.5_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_claude35), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

df['diag_spec_Claude3_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_claude3), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

df['diag_spec_Haiku_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_haiku), axis=1)
df.to_csv('MIMIC-IV-Ext-Diagnosis-Specialty.csv', index=False)

## LLM Prediction Triage General User Case

In [ ]:
## Load Data from create_ground_truth_specialty.py
df = pd.read_csv("MIMIC-IV-Ext-Triage.csv")

In [ ]:
## Define the prompt template
prompt = """You are a nurse with emergency and triage experience. Using the patient's history of present illness and his information, determine the triage level based on the Emergency Severity Index (ESI), ranging from ESI level 1 (highest acuity) to ESI level 5 (lowest acuity): 1: Assign if the patient requires immediate lifesaving intervention. 2: Assign if the patient is in a high-risk situation (e.g., confused, lethargic, disoriented, or experiencing severe pain/distress)  3: Assign if the patient requires two or more diagnostic or therapeutic interventions and their vital signs are within acceptable limits for non-urgent care. 4: Assign if the patient requires one diagnostic or therapeutic intervention (e.g., lab test, imaging, or EKG). 5: Assign if the patient does not require any diagnostic or therapeutic interventions beyond a physical exam (e.g., no labs, imaging, or wound care).
History of present illness: {HPI} and patient info: {patient_info}. Respond with the level in an <acuity> tag."""


## set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"]="Enter your AWS Access Key ID"
os.environ["AWS_SECRET_ACCESS_KEY"]="Enter your AWS Secret Access Key"

prompt_chain = PromptTemplate(template=prompt,input_variables=["hpi", "patient_info"])
client = boto3.client(service_name="bedrock-runtime", region_name=str("us-east-1"))


## Claude Sonnet 3.5
llm_claude35 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={"temperature": 0},client=client)
chain_claude35 = prompt_chain | llm_claude35


## Claude Sonnet 3
llm_claude3 = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_claude3 = prompt_chain | llm_claude3


## Claude 3 Haiku
llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_haiku = prompt_chain | llm_haiku


tqdm.pandas()
df['triage_Claude3.5'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_claude35), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)

df['triage_Claude3'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_claude3), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)

df['triage_Haiku'] = df.progress_apply(lambda row: get_prediction_GeneralUser(row, chain_haiku), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)

## LLM Prediction Triage Clinical User Case

In [ ]:
## Define the prompt template
prompt = """You are a nurse with emergency and triage experience. Using the patient's history of present illness, his information and initial vitals, determine the triage level based on the Emergency Severity Index (ESI), ranging from ESI level 1 (highest acuity) to ESI level 5 (lowest acuity): 1: Assign if the patient requires immediate lifesaving intervention. 2: Assign if the patient is in a high-risk situation (e.g., confused, lethargic, disoriented, or experiencing severe pain/distress)  3: Assign if the patient requires two or more diagnostic or therapeutic interventions and their vital signs are within acceptable limits for non-urgent care. 4: Assign if the patient requires one diagnostic or therapeutic intervention (e.g., lab test, imaging, or EKG). 5: Assign if the patient does not require any diagnostic or therapeutic interventions beyond a physical exam (e.g., no labs, imaging, or wound care).
History of present illness: {HPI}, patient info: {patient_info} and initial vitals: {initial_vitals}. Respond with the level in an <acuity> tag."""


## set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"]="Enter your AWS Access Key ID"
os.environ["AWS_SECRET_ACCESS_KEY"]="Enter your AWS Secret Access Key"

prompt_chain = PromptTemplate(template=prompt,input_variables=["hpi", "patient_info", "initial_vitals"])
client = boto3.client(service_name="bedrock-runtime", region_name=str("us-east-1"))


## Claude Sonnet 3.5
llm_claude35 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={"temperature": 0},client=client)
chain_claude35 = prompt_chain | llm_claude35


## Claude Sonnet 3
llm_claude3 = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_claude3 = prompt_chain | llm_claude3


## Claude 3 Haiku
llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",  model_kwargs={"temperature": 0},client=client)
chain_haiku = prompt_chain | llm_haiku


tqdm.pandas()
df['triage_Claude3.5_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_claude35), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)

df['triage_Claude3_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_claude3), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)

df['triage_Haiku_Clinical'] = df.progress_apply(lambda row: get_prediction_ClinicalUser(row, chain_haiku), axis=1)
df.to_csv('MIMIC-IV-Ext-Triage.csv', index=False)